In [29]:
import torch
import torch.nn as nn
import numpy as np

class ResidualUnit(nn.Module):
    def __init__(self, in_channels, use_bn = False):
        super().__init__()
        self.use_bn = use_bn
        if self.use_bn:
            self.bn1 = nn.BatchNorm2d(in_channels)
            self.bn2 = nn.BatchNorm2d(in_channels)    
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=1, padding=1, bias=True)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU() # maybe not be required

    def forward(self, x):
        residual = x
        out = x
        if self.use_bn:
            out = self.bn1(x)
        out = self.relu1(out)
        out = self.conv1(out)
        if self.use_bn:
            out = self.bn2(out)
        out = self.relu2(out)
        out = self.conv2(out)
        out += residual
        return out

class ResidualPipeline(nn.Module):
    def __init__(self, in_channels, n_units, use_bn = False):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, 64, kernel_size=3, stride=1, padding=1, bias=True)
        self.conv2 = nn.Conv2d(64, 2, kernel_size=3, stride=1, padding=1, bias=True)
        layers = []
        for i in range(n_units):
            layers.append(ResidualUnit(64, use_bn))
        self.resnet_stack = nn.Sequential(*layers)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.resnet_stack(x)
        x = self.relu(x) # not there in original paper
        x = self.conv2(x)
        return x

class ExternalEncoder(nn.Module):
    def __init__(self, ext_dim, map_dim):
        super().__init__()
        self.map_dim = map_dim
        self.fcc1 = nn.Linear(ext_dim, 64, bias = True)
        self.fcc2 = nn.Linear(64, np.prod(map_dim), bias = True)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU() # maybe not be required
        
    def forward(self, x):
        x = self.fcc1(x)
        x = self.relu1(x)
        x = self.fcc2(x)
        x = self.relu2(x)
        return x.view(-1, *self.map_dim)
    
class STResnet(nn.Module):
    def __init__(self, c_channel, p_channel, t_channel, n_residual_units, ext_dim, map_dim, use_bn = False):
        super().__init__()
        self.e_pipe = ExternalEncoder(ext_dim, map_dim)
        self.c_pipe = ResidualPipeline(c_channel, n_residual_units, use_bn)
        self.p_pipe = ResidualPipeline(p_channel, n_residual_units, use_bn)
        self.t_pipe = ResidualPipeline(t_channel, n_residual_units, use_bn)
        # 1 dimension for batch processing, this class cannot process unbatched data
        self.w_c = nn.Parameter(torch.randn(1, *map_dim))
        self.w_p = nn.Parameter(torch.randn(1, *map_dim))
        self.w_t = nn.Parameter(torch.randn(1, *map_dim))
        self.tanh = nn.Tanh()
        
    def forward(self, x_c, x_p, x_t, x_e):
        y_e = self.e_pipe(x_e)
        y_c = self.c_pipe(x_c)
        y_p = self.p_pipe(x_p)
        y_t = self.t_pipe(x_t)
        # Fusion: Eliment wise product (Hadamard Product)
        y = self.w_c*y_c + self.w_p*y_p + self.w_t*y_t
        y = self.tanh(y+y_e)
        return y

modelpath = './checkpoints/TaxiBJ/k_fold_stresnet.pt'
if not torch.cuda.is_available():
    model = torch.load(modelpath, map_location=torch.device('cpu'))
else:
    model = torch.load(modelpath, map_location=torch.device('cuda'))
    torch.cuda.get_device_name(0)
model.eval();


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Colab Notebooks/TaxiBJ


STResnet(
  (e_pipe): ExternalEncoder(
    (fcc1): Linear(in_features=27, out_features=64, bias=True)
    (fcc2): Linear(in_features=64, out_features=2048, bias=True)
    (relu1): ReLU()
    (relu2): ReLU()
  )
  (c_pipe): ResidualPipeline(
    (conv1): Conv2d(8, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (resnet_stack): Sequential(
      (0): ResidualUnit(
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu1): ReLU()
        (relu2): ReLU()
      )
      (1): ResidualUnit(
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (bn2):

'Tesla T4'

In [30]:
from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from TaxiBJ import TaxiBJDataset
from torch.utils.data import DataLoader
import math

n_closeness = 4 # 4*30 minutes
n_period = 1 # 1 day
n_trend = 1 # 1 week
taxibj_dataset_test = TaxiBJDataset('./Datasets/TaxiBJ/',n_closeness,n_period,n_trend,0.001,False)
mmn = taxibj_dataset_test.mmn_flow

min: 0.0 max: 1292.0
min: -15.1 max: 36.1
min: 0.0 max: 31.32


In [43]:
from tqdm import tqdm
# t step ahead prediction calculate loss
t = 4
t_ = 4
batch_size = 1
test_loader = DataLoader(taxibj_dataset_test, batch_size=batch_size)
loss_fun = torch.nn.MSELoss(reduction = 'sum')
tstep_data = [None]*t
replace = []  # stores t step ahead predictions for replacement
tstep_loss = 0.0
for x_c, x_p, x_t, x_e, y in tqdm(test_loader):
    with torch.no_grad():
        if t_ == 0:
            x_c[0, -t*2:] = torch.vstack(replace) # replace the appropriate closeness with previous iterations' predictions data
            y_pred_t = model(x_c, x_p, x_t, x_e) # make tstep prediction
            tstep_data.append(y_pred_t)
            loss = loss_fun(mmn.inverse_transform(y_pred_t), mmn.inverse_transform(y))
            tstep_loss += loss.item()
            replace.append(torch.squeeze(y_pred_t, 0)) # (1, 2, 32, 32) -> (2, 32, 32)
            if len(replace) > t: del replace[0] # remove time that is more than past 2 hours ago (n_closeness=4)
        else:
            y_pred = model(x_c, x_p, x_t, x_e)
            replace.append(torch.squeeze(y_pred, 0)) # (1, 2, 32, 32) -> (2, 32, 32)
            t_ -= 1

100%|██████████| 19997/19997 [04:21<00:00, 76.47it/s]


In [44]:
# output data
# InteractiveShell.ast_node_interactivity = "all"
# len(tstep_data)

# tstep_data1 = tstep_data.copy()
# tstep_data1 = [data.detach().cpu().numpy() for data in tstep_data1 if data != None]

# tstep_data2 = tstep_data.copy()
# tstep_data2 = [data.detach().cpu().numpy() for data in tstep_data2 if data != None]

# tstep_data3 = tstep_data.copy()
# tstep_data3 = [data.detach().cpu().numpy() for data in tstep_data3 if data != None]

tstep_data4 = tstep_data.copy()
tstep_data4 = [data.detach().cpu().numpy() for data in tstep_data4 if data != None]

np.save(f'./Datasets/TaxiBJ/TaxiBJ_{t}step', eval(f'tstep_data{t}'))

In [45]:
rmse_loss = math.sqrt(tstep_loss/(len(test_loader.sampler)-t)) # first t predictions are used to train for the t+1 prediction
print('Map Root Mean Square Loss: ', rmse_loss)

map_dim = (2, 32, 32) # Specific to dataset
rmse_loss = math.sqrt(tstep_loss/((len(test_loader.sampler)-t)*np.prod(map_dim)))
print('Grid Root Mean Square Loss: ', rmse_loss)

Map Root Mean Square Loss:  4668.296179204055
Grid Root Mean Square Loss:  103.15574640320118
